# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
# load messages dataset
messages = pd.read_csv('disaster_messages.csv', encoding='latin-1')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
messages.describe(include='all')

,id,message,original,genre
count,26248.000000,26248,10184,26248
unique,NaN,26177,9630,3
top,NaN,#NAME?,Nap fe ou konnen ke apati de jodi a sevis SMS ...,news
freq,NaN,4,20,13068
mean,15224.078368,NaN,NaN,NaN
std,8826.069156,NaN,NaN,NaN
min,2.000000,NaN,NaN,NaN
25%,7445.750000,NaN,NaN,NaN
50%,15660.500000,NaN,NaN,NaN
75%,22923.250000,NaN,NaN,NaN


In [4]:
# load categories dataset
categories = pd.read_csv('disaster_categories.csv', encoding='latin-1')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
categories.describe(include='all')

,id,categories
count,26248.000000,26248
unique,NaN,4003
top,NaN,related-0;request-0;offer-0;aid_related-0;medi...
freq,NaN,6125
mean,15224.078368,NaN
std,8826.069156,NaN
min,2.000000,NaN
25%,7445.750000,NaN
50%,15660.500000,NaN
75%,22923.250000,NaN


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
df = messages.merge(categories, how='inner', on=["id"])
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
df.describe(include='all')

,id,message,original,genre,categories
count,26386.000000,26386,10246,26386,26386
unique,NaN,26177,9630,3,4003
top,NaN,Shelter materials (thick polyesters) are being...,Nap fe ou konnen ke apati de jodi a sevis SMS ...,news,related-0;request-0;offer-0;aid_related-0;medi...
freq,NaN,9,20,13128,6140
mean,15217.885886,NaN,NaN,NaN,NaN
std,8823.741128,NaN,NaN,NaN,NaN
min,2.000000,NaN,NaN,NaN,NaN
25%,7438.250000,NaN,NaN,NaN,NaN
50%,15650.500000,NaN,NaN,NaN,NaN
75%,22916.750000,NaN,NaN,NaN,NaN


## So the row count=26386 in the merged df is more than messages and categories, 26248

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [8]:
# create a dataframe of the 36 individual category columns
categories = df["categories"].str.split(";", expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [9]:
categories.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
count,26386,26386,26386,26386,26386,26386,26386,26386,26386,26386,...,26386,26386,26386,26386,26386,26386,26386,26386,26386,26386
unique,3,2,2,2,2,2,2,2,2,1,...,2,2,2,2,2,2,2,2,2,2
top,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
freq,20042,21873,26265,15432,24287,25067,25661,25915,25523,26386,...,26077,25231,19043,24209,23922,26104,23925,25853,25007,21273


In [10]:
cats = categories.describe(include='all')
for i in range(cats.shape[1]):
    print(cats.iloc[:, i])

count         26386
unique            3
top       related-1
freq          20042
Name: 0, dtype: object
count         26386
unique            2
top       request-0
freq          21873
Name: 1, dtype: object
count       26386
unique          2
top       offer-0
freq        26265
Name: 2, dtype: object
count             26386
unique                2
top       aid_related-0
freq              15432
Name: 3, dtype: object
count              26386
unique                 2
top       medical_help-0
freq               24287
Name: 4, dtype: object
count                  26386
unique                     2
top       medical_products-0
freq                   25067
Name: 5, dtype: object
count                   26386
unique                      2
top       search_and_rescue-0
freq                    25661
Name: 6, dtype: object
count          26386
unique             2
top       security-0
freq           25915
Name: 7, dtype: object
count          26386
unique             2
top       military-0
freq 

## Notice the related category has three unique values and child_alone has only one.

In [11]:
# select the first row of the categories dataframe
row = categories.iloc[0]
print("Raw categories\n", row)
# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x: x[0:-2])
print("Cleaned Category names\n", category_colnames)

Raw categories
 0                    related-1
1                    request-0
2                      offer-0
3                aid_related-0
4               medical_help-0
5           medical_products-0
6          search_and_rescue-0
7                   security-0
8                   military-0
9                child_alone-0
10                     water-0
11                      food-0
12                   shelter-0
13                  clothing-0
14                     money-0
15            missing_people-0
16                  refugees-0
17                     death-0
18                 other_aid-0
19    infrastructure_related-0
20                 transport-0
21                 buildings-0
22               electricity-0
23                     tools-0
24                 hospitals-0
25                     shops-0
26               aid_centers-0
27      other_infrastructure-0
28           weather_related-0
29                    floods-0
30                     storm-0
31                     

In [12]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [13]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].astype("str").apply(
        lambda x: x[-1])
    # convert column from string to numeric
    categories[column] = categories[column].astype("int32")

categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# let's look at the summary stats for the categories
categories.describe(include='all')

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26386.000000,26386.000000,26386.000000,26386.000000,26386.00000,26386.000000,26386.000000,26386.00000,26386.000000,26386.0,...,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000
mean,0.775032,0.171038,0.004586,0.415144,0.07955,0.049989,0.027477,0.01785,0.032707,0.0,...,0.011711,0.043773,0.278292,0.082506,0.093383,0.010687,0.093269,0.020200,0.052263,0.193777
std,0.435692,0.376549,0.067564,0.492756,0.27060,0.217926,0.163471,0.13241,0.177871,0.0,...,0.107583,0.204594,0.448166,0.275139,0.290974,0.102828,0.290815,0.140687,0.222560,0.395264
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
# Above result rows, there is one more than expected 0 and 1! may be we have to clean it up later
categories["related"].unique()

array([1, 0, 2])

In [16]:
# let's count them
categories.groupby(by=["related"]).count()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140,...,6140,6140,6140,6140,6140,6140,6140,6140,6140,6140
1,20042,20042,20042,20042,20042,20042,20042,20042,20042,20042,...,20042,20042,20042,20042,20042,20042,20042,20042,20042,20042
2,204,204,204,204,204,204,204,204,204,204,...,204,204,204,204,204,204,204,204,204,204


In [17]:
# Above result rows, there is ony 0 for child_alone
categories["child_alone"].unique()

array([0])

In [18]:
# let's count them
categories.groupby(by=["child_alone"]).count()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
child_alone,,,,,,,,,,,,,,,,,,,,,
0,26386,26386,26386,26386,26386,26386,26386,26386,26386,26386,...,26386,26386,26386,26386,26386,26386,26386,26386,26386,26386


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [19]:
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# drop the original categories column from `df`
df.drop(["categories"], axis=1, inplace=True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [21]:
# Concatenate df and categories data frames.
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# let's check new df summary stats
df.describe(include='all')

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26386.000000,26386,10246,26386,26386.000000,26386.000000,26386.000000,26386.000000,26386.00000,26386.000000,...,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000
unique,NaN,26177,9630,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Shelter materials (thick polyesters) are being...,Nap fe ou konnen ke apati de jodi a sevis SMS ...,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,9,20,13128,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,15217.885886,NaN,NaN,NaN,0.775032,0.171038,0.004586,0.415144,0.07955,0.049989,...,0.011711,0.043773,0.278292,0.082506,0.093383,0.010687,0.093269,0.020200,0.052263,0.193777
std,8823.741128,NaN,NaN,NaN,0.435692,0.376549,0.067564,0.492756,0.27060,0.217926,...,0.107583,0.204594,0.448166,0.275139,0.290974,0.102828,0.290815,0.140687,0.222560,0.395264
min,2.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7438.250000,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15650.500000,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22916.750000,NaN,NaN,NaN,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
# Let's verify the child_alone has only zeros and then drop the column
df["child_alone"].unique()

array([0])

In [24]:
# drop the column child_alone
df.drop(labels=["child_alone"], axis=1, inplace=True)

In [25]:
# Verify child_alone column is dropped
var = 'child_alone'
if var in list(df.columns):
    print("child_alone not dropped")
else:
    print("column child_alone dropped")

column child_alone dropped


In [26]:
# Drop the rows with related = 2. First get the row numbers where related =2
related2_rownum = df[(df['related'] == 2)].index
# verify
df.iloc[related2_rownum]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
221,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
462,565,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
578,700,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20465,23662,transferred to a sanitary landfill site by a,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20636,23861,Families also have solar lamps which can be re...,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22481,25874,Read the [full blog post](http://www.odi.org.u...,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23537,27071,Actualmente e independientemente de la ayuda d...,NaN,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# drop those rows from the dataframe
df.drop(related2_rownum, inplace=True)

In [28]:
# Let's verify the related column has only 0 and 1
df["related"].unique()

array([1, 0])

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [29]:
# check number of duplicates
print("Total rows=", df.count())
print("% duplicate rows=", df.duplicated(subset=None, keep='first').mean())

Total rows= id                        26182
message                   26182
original                  10099
genre                     26182
related                   26182
request                   26182
offer                     26182
aid_related               26182
medical_help              26182
medical_products          26182
search_and_rescue         26182
security                  26182
military                  26182
water                     26182
food                      26182
shelter                   26182
clothing                  26182
money                     26182
missing_people            26182
refugees                  26182
death                     26182
other_aid                 26182
infrastructure_related    26182
transport                 26182
buildings                 26182
electricity               26182
tools                     26182
hospitals                 26182
shops                     26182
aid_centers               26182
other_infrastructure      26

In [30]:
def remove_duplicates_from_df(df):
    """
    This function takes a dataframe as input and removes any duplicates from the dataframe.
    Returns the dataframe with duplicates removed, and confirmation message indicating no. of duplicates removed.
    
    Parameters:
    df (dataframe): The dataframe from which duplicates should be removed.
    
    Returns:
    dataframe, str: The dataframe with duplicates removed, and message indicating no. of duplicates removed
    """
    original_row_count = df.shape[0]
    df.drop_duplicates(inplace=True)
    new_row_count = df.shape[0]
    duplicates_removed = original_row_count - new_row_count
    if duplicates_removed == 0:
        return df, f"No duplicates found in the dataframe."
    else:
        return df, f"{duplicates_removed} duplicates removed from the dataframe."

In [31]:
# drop duplicate rows
df, dupknt = remove_duplicates_from_df(df)
print(dupknt)

154 duplicates removed from the dataframe.


In [32]:
# check number of duplicate rows after
print("Total rows=\n", df.count())
print("% duplicate rows=", df.duplicated(subset=None, keep='first').mean())

Total rows=
 id                        26028
message                   26028
original                  10038
genre                     26028
related                   26028
request                   26028
offer                     26028
aid_related               26028
medical_help              26028
medical_products          26028
search_and_rescue         26028
security                  26028
military                  26028
water                     26028
food                      26028
shelter                   26028
clothing                  26028
money                     26028
missing_people            26028
refugees                  26028
death                     26028
other_aid                 26028
infrastructure_related    26028
transport                 26028
buildings                 26028
electricity               26028
tools                     26028
hospitals                 26028
shops                     26028
aid_centers               26028
other_infrastructure      2

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [33]:
# First check if the DB file exists, and if it does, delete it before creating the SQLite engine.
if os.path.exists("DisasterResponse.db"):
    os.remove("DisasterResponse.db")

# Create the SQLite engine
engine = create_engine('sqlite:///DisasterResponse.db')

# copy the df dataframe to SQLITE db
# df.to_sql('DisasterResponse', engine, index=False)
#
database_filename = 'DisasterResponse.db'
tablename = database_filename.split('.')[0]
print(tablename)
df.to_sql(tablename, engine, index=False)

DisasterResponse


26028

In [34]:
# validate the records are wrtten to the database
print("dateframe has rows:  ", df.shape[0])
numrows = pd.read_sql("SELECT COUNT(*) as rows FROM DisasterResponse", engine)
print("Db table has ", numrows.iloc[0])

dateframe has rows:   26028
Db table has  rows    26028
Name: 0, dtype: int64


In [35]:
## valiedate the contents of the database table
import sqlite3

con = sqlite3.connect("DisasterResponse.db")
cursor = con.cursor()
## cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

cursor.execute(
    "SELECT related, count(*) FROM DisasterResponse group by related;")
res = cursor.fetchall()

for tb in res:
    print(tb)

con.close()

(0, 6122)
(1, 19906)


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.